In [69]:
#!pip install flask
from flask import Flask, request, render_template
from tensorflow.keras.models import load_model
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [70]:
app = Flask(__name__)


In [71]:
# Load NLTK resources
nltk.download('punkt')
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ADMIN\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ADMIN\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [72]:
# Load model and tokenizer
model = load_model('sentiment_model.h5')

import pickle

# Load tokenizer
with open("tokenizer.pickle", "rb") as handle:
    tokenizer = pickle.load(handle)


In [73]:
with open("sequence_length.txt", "r") as f:
    sequence_length = int(f.read())

In [ ]:

# Routes
@app.route('/', methods=['GET', 'POST'])
def index():
    sentiment = None
    user_review = ""
    
    if request.method == 'POST':
        user_review = request.form.get('review', "").strip()

        if user_review:  # only process if not empty
            cleaned_text = clean_text(user_review)
            sequence = tokenizer.texts_to_sequences([cleaned_text])
            padded = pad_sequences(sequence, maxlen=sequence_length, padding='post')
            prediction = model.predict(padded)[0][0]
            sentiment = 'Positive review' if prediction > 0.5 else 'Negative review'
        else:
            sentiment = "Please enter a review."

    return render_template('index.html', sentiment=sentiment, review=user_review)

if __name__ == '__main__':
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat


SystemExit: 1